<a href="https://colab.research.google.com/github/gpandu/CodeGenGPT/blob/main/CodeGPT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training GPT model is done using following steps.
1. Load and prepare dataset for tokenizer training.
2. Train BPE tokenizer from scratch.
3. Prepare tensorflow dataset with generator.
4. Create layer and model classes for GPT



*   We will use tokenizers library from HuggingFace to train tokenizer from scratch.
*   We will also use datasets to load the python "code_search_net" dataset. It has ~410k of training records. If we load the load at once we will run out of RAM, so we will take advantage streaming the batches.




In [1]:
!pip install tokenizers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.3 MB/s eta 0:00:00


In [2]:
import math
import numpy as np
import tensorflow as tf
from datasets import load_dataset

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


*   Load the "code_search_net" adn we can check details of the dataset like below.



In [3]:
# load code dataset
raw_dataset = load_dataset("code_search_net", "python")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [4]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 412178
    })
    test: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 22176
    })
    validation: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 23107
    })
})



*   Create a generator to load the data in batches for training tokenizer.

   >  Loading all the data at once may cause out of memory error.







In [5]:
tokenizer_batch_len = 1000
def get_training_corpus():
    dataset = raw_dataset["train"]
    for start_idx in range(0, len(dataset), tokenizer_batch_len):
        samples = dataset[start_idx : start_idx + tokenizer_batch_len]
        yield samples["whole_func_string"]


In [6]:
#check if we are able to iterate over the dataset.
iterat = iter(get_training_corpus())
next(iterat)[10]

'def leaveEvent(self, event):\n        """\n        Reimplements the :meth:`QLabel.leaveEvent` method.\n\n        :param event: QEvent.\n        :type event: QEvent\n        """\n\n        if self.__checkable:\n            not self.__checked and self.setPixmap(self.__default_pixmap)\n        else:\n            self.setPixmap(self.__default_pixmap)'

Tokenization:



*   Subword Tokenization : Keep frequent words and break rearer words into subwords
*   A statastical Alogrothm learns how to do this based on corpus.

> Ex: Listeria ---> "more" , "over"

> "more" and "over" are likely to be more frequent than moreover


*   Tokenization has better chance of handling OOV words while decreasing the size of the overall dictionary.   

* We will use BPE(Byte Pair Encoding) to train tokenizer on "code_search_net" python Dataset.
*   For more information on BPE can be found here. https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt


In [7]:
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

# "<|endoftext|>" will used to stop the sequence generation during inference. This is also
#  a way telling GPT to learn to about the end of the sequence
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["PAD","<|endoftext|>"])
#Train the tokenizer using BPE trainer, loads the data in batches
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [8]:
tokenizer.save("tokenizer.json")


*    Check if we are able to tokenize and encode the data using trained BPE tokenizer
*   Actual maximum sequence length MAX_SEQ_LENGTH = 256, we will add one to 256 so that last sample will dropped for the inputs and first sample will be dropped from the outputs. This makes model to see only previous samples to predict next sample.

        For example:
                 input  :     295    4354   63    72      6035   63
                 output :     4354   63     72    6035    63     3170




In [9]:
context_size = 256
tokenizer.enable_padding(length=context_size+1, pad_id = 0, pad_token = "PAD")
tokenizer.enable_truncation(max_length=context_size+1)
#tokenizer.post_process()
tokenizer.decoder = decoders.BPEDecoder()
encoding = tokenizer.encode(raw_dataset["train"][50]["whole_func_string"])
print(encoding.ids)
print(encoding.attention_mask)

[296, 301, 64, 420, 64, 2581, 64, 1300, 9, 1942, 420, 275, 233, 291, 223, 4952, 232, 662, 13, 1589, 556, 312, 257, 662, 301, 232, 2269, 1341, 386, 301, 4523, 223, 232, 2269, 1341, 15, 233, 291, 233, 257, 420, 234, 662, 15, 10316, 9, 1942, 420, 10, 313, 265, 662, 15, 4300, 9, 1942, 420, 275, 223, 1023, 13, 1289, 234, 662, 15, 6065, 9, 1942, 420, 10, 223, 265, 1289, 254, 410, 598, 64, 18402, 27, 242, 303, 624, 223, 303, 694, 313, 265, 662, 15, 4894, 9, 1942, 420, 275, 223, 297, 3657, 254, 410, 598, 64, 18402, 27, 242, 265, 662, 15, 4300, 9, 420, 15, 778, 9, 1942, 420, 13, 4489, 1864, 613, 6, 84, 8, 447, 3657, 1570, 285, 303, 624, 233, 303, 694, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0



*   Generator to prepare inputs and outputs in the batches.
*   Inputs and Outputs will have sequences ids encoded from the tokenizer.



In [10]:
batch_size = 100
def generate_train_data():
  decoder_inputs = []
  decoder_targets = []
  dataset = raw_dataset["train"]
  for start_idx in range(0, len(dataset)-(len(dataset)%batch_size), batch_size):
      samples = dataset[start_idx : start_idx + batch_size]
      seqs = tokenizer.encode_batch(samples["whole_func_string"])
      decoder_inputs = [seq.ids[:-1] for seq in seqs] # Drop the last token in the sentence.
      decoder_targets = [seq.ids[1:] for seq in seqs]  # Drop the first token in the sentence.
      yield decoder_inputs, decoder_targets


In [ ]:
#tf_dataset = raw_dataset["train"].to_tf_dataset(batch_size = 100, columns = ['whole_func_string'])
#tf_dataset

<_PrefetchDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>

In [11]:
iterator = iter(generate_train_data())
decoder_inputs, decoder_targets = next(iterator)
print(decoder_inputs[80])


[296, 1621, 9, 17130, 11493, 13, 1167, 64, 1125, 13, 1167, 64, 2777, 7656, 1790, 64, 363, 5853, 7656, 881, 634, 328, 1054, 3659, 1263, 30, 723, 13, 6793, 30, 723, 275, 233, 291, 233, 10313, 257, 869, 1167, 64, 1125, 498, 19248, 11493, 351, 1484, 8047, 1661, 297, 233, 2738, 2784, 821, 12136, 309, 1256, 1481, 1661, 603, 1488, 265, 1167, 64, 1125, 7127, 254, 233, 257, 15616, 10, 508, 8047, 1661, 4487, 682, 1167, 2223, 312, 233, 257, 1663, 312, 257, 15616, 443, 2223, 5954, 1790, 64, 1125, 603, 4272, 15, 929, 552, 233, 1540, 1790, 64, 1125, 6779, 1167, 1523, 293, 232, 496, 312, 2223, 948, 603, 233, 4272, 15, 648, 1790, 64, 363, 5853, 301, 340, 2040, 578, 1025, 1850, 1736, 1523, 293, 233, 20211, 948, 665, 372, 2073, 9454, 257, 1736, 449, 1907, 12581, 2991, 233, 257, 1736, 456, 15, 233, 508, 8047, 1661, 8815, 1369, 2943, 2288, 293, 496, 312, 8047, 233, 1661, 259, 297, 1099, 3894, 462, 257, 869, 1167, 456, 233, 259, 358, 1687, 64, 1672, 64, 325, 359, 234, 405, 749, 467, 17, 13, 757, 467, 18, 1

In [12]:
#tf_dataset = tf.data.Dataset.from_tensor_slices((decoder_inputs, decoder_targets)).batch(10, drop_remainder=True)

tf_dataset = tf.data.Dataset.from_generator(generate_train_data, output_types=(tf.int32, tf.int32), output_shapes=(tf.TensorShape([batch_size, context_size]), tf.TensorShape([batch_size, context_size])))

# checks to see if data is loading properly.
iterator = iter(tf_dataset)
ins, outs = next(iterator)
print(ins.shape)
next(iterator)[0]

(100, 256)


<tf.Tensor: shape=(100, 256), dtype=int32, numpy=
array([[  296,  8118,    64, ...,     0,     0,     0],
       [  296,  8118,    64, ...,  7171,     9,   831],
       [  296,  8118,    64, ...,     0,     0,     0],
       ...,
       [  296,   754,     9, ...,     0,     0,     0],
       [  296, 19549,    64, ...,     0,     0,     0],
       [  296,  1621,    64, ...,     0,     0,     0]], dtype=int32)>

Multi Head Attention



*   Each Attention head performs Scaled Dot Product Self-Attention operation where given Keys, Query and Values, the return matrix of values given by below operation.

        Attention(Q,K,V) = softmax((Q*Transpose(K))/sqrt(d))*V





In [13]:
def scaled_dot_product_attention(query, key, value, mask=None):
  key_dims = tf.cast(tf.shape(key)[-1], tf.float32)
  scaled_scores = tf.matmul(query, key, transpose_b=True) / tf.math.sqrt(key_dims)

  if mask is not None:
    scaled_scores = tf.where(mask==0, -np.inf, scaled_scores)

  softmax = tf.keras.layers.Softmax()
  weights = softmax(scaled_scores)
  return tf.matmul(weights, value), weights



**Generating queries, keys, and values for multiple heads.**

> Now that we have a way to calculate self-attention, let's actually generate the input queries, keys, and values for multiple heads.

>  each attention head had its own separate set of query, key, and value weights. Each weight matrix was of dimension  d x d/h  where h was the number of heads.




In [14]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.d_model = d_model
    self.num_heads = num_heads

    self.d_head = self.d_model // self.num_heads

    self.w_queries = tf.keras.layers.Dense(self.d_model, use_bias=False)
    self.w_keys = tf.keras.layers.Dense(self.d_model, use_bias=False)
    self.w_values = tf.keras.layers.Dense(self.d_model, use_bias=False)

    # Linear layer to generate the final output.
    self.dense = tf.keras.layers.Dense(self.d_model)

  def split_heads(self, x):
    batch_size = x.shape[0]

    split_inputs = tf.reshape(x, (batch_size, -1, self.num_heads, self.d_head))
    return tf.transpose(split_inputs, perm=[0, 2, 1, 3])

  def merge_heads(self, x):
    batch_size = x.shape[0]

    merged_inputs = tf.transpose(x, perm=[0, 2, 1, 3])
    return tf.reshape(merged_inputs, (batch_size, -1, self.d_model))

  def call(self, query, key, value, mask):
    queries = self.w_queries(query)
    keys = self.w_keys(key)
    values = self.w_values(value)

    queries = self.split_heads(queries)
    keys = self.split_heads(keys)
    values = self.split_heads(values)

    output, attn_weights = scaled_dot_product_attention(queries, keys, values, mask)
    output = self.merge_heads(output)

    return self.dense(output), attn_weights


Feed Forward Neural Network

In [15]:
def feed_forward_network(d_model, hidden_dim):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(hidden_dim, activation='relu'),
      tf.keras.layers.Dense(d_model)
  ])

Decode Block

In [16]:
class DecoderBlock(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, hidden_dim, dropout_rate=0.1):
    super(DecoderBlock, self).__init__()

    self.mhsa1 = MultiHeadAttention(d_model, num_heads)

    self.ffn = feed_forward_network(d_model, hidden_dim)

    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    self.layernorm1 = tf.keras.layers.LayerNormalization()
    self.layernorm2 = tf.keras.layers.LayerNormalization()

  def call(self, input, training, decoder_mask):
    mhsa_output1, attn_weights = self.mhsa1(input, input, input, decoder_mask)
    mhsa_output1 = self.dropout1(mhsa_output1, training=training)
    mhsa_output1 = self.layernorm1(mhsa_output1 + input)

    ffn_output = self.ffn(mhsa_output1)
    ffn_output = self.dropout2(ffn_output, training=training)
    output = self.layernorm2(ffn_output + mhsa_output1)

    return output, attn_weights


Decoder with Mulitple Layers

In [17]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
               max_seq_len, dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.max_seq_len = max_seq_len

    self.token_embed = tf.keras.layers.Embedding(target_vocab_size, self.d_model)
    self.pos_embed = tf.keras.layers.Embedding(max_seq_len, self.d_model)

    self.dropout = tf.keras.layers.Dropout(dropout_rate)

    self.blocks = [DecoderBlock(self.d_model, num_heads, hidden_dim, dropout_rate) for _ in range(num_blocks)]

  def call(self, input, training, decoder_mask):
    token_embeds = self.token_embed(input)

    seq_len = input.shape[1]
    # Generate position indices.
    num_pos = input.shape[0] * seq_len
    pos_idx = np.resize(np.arange(seq_len), num_pos)
    pos_idx = np.reshape(pos_idx, input.shape)

    pos_embeds = self.pos_embed(pos_idx)

    x = self.dropout(token_embeds + pos_embeds, training=training)

    for block in self.blocks:
      x, weights = block(x, training, decoder_mask)

    return x, weights

Custom loss function to remove effect of padding

In [18]:
def loss_func(targets, logits):
  ce_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  mask = tf.cast(tf.math.not_equal(targets, 0), tf.float32)
  return ce_loss(targets, logits, sample_weight=mask)

In [19]:
class GPTModel(tf.keras.Model):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
               max_input_len, dropout_rate=0.1):
    super(GPTModel, self).__init__()

    self.decoder = Decoder(num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
                           max_input_len, dropout_rate)

    # The final dense layer to generate logits from the model output.
    self.output_layer = tf.keras.layers.Dense(target_vocab_size)

  @tf.function
  def train_step(self, inputs):
      loss = 0.

      input_seq, targets = inputs
      with tf.GradientTape() as tape:

        dec_padding_mask = tf.cast(tf.math.not_equal(input_seq, 0), tf.float32)
        dec_padding_mask = dec_padding_mask[:, tf.newaxis, tf.newaxis, :]
        input_seq_len = len(input_seq[0])
        look_ahead_mask = tf.linalg.band_part(tf.ones((input_seq_len,
                                               input_seq_len)), -1, 0)
        dec_mask = tf.minimum(dec_padding_mask, look_ahead_mask)

        logits, _ = self.decoder(input_seq, True, dec_mask)
        logits =   self.output_layer(logits)
        loss += self.loss(targets, logits)

      # Update the parameters and the optimizer
      variables = self.decoder.trainable_variables
      gradients = tape.gradient(loss, variables)
      self.optimizer.apply_gradients(zip(gradients, variables))

      return {'loss': loss}

  def call(self, input, training):
    logits, _ = self.decoder(input, False, None)
    logits =   self.output_layer(logits)
    return logits



In [20]:
tf.keras.utils.set_random_seed(10)

In [21]:
model = GPTModel(
    num_blocks = 6,
    d_model = 512,
    num_heads = 4,
    hidden_dim = 1024,
    target_vocab_size = tokenizer.get_vocab_size(),
    max_input_len = 256)

optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss_func, run_eagerly=True)

In [ ]:
epochs = 2
model.fit(tf_dataset, epochs=epochs)

Epoch 1/2
4121/4121 [==============================] - 5597s 1s/step - loss: 5.7481
Epoch 2/2
3887/4121 [===========================>..] - ETA: 5:09 - loss: 5.4907

In [ ]:
def top_k_logits(logits, k):
	if k == 0:
		return logits

	values, _ = tf.nn.top_k(logits, k=k)
	min_values = values[:, -1]

	return tf.where(logits < min_values, tf.ones_like(logits, dtype=logits.dtype) * -1e10, logits)

# Nucleas Sampling (https://arxiv.org/pdf/1904.09751.pdf)


def top_p_logits(logits, p):
	"""Took from OpenAI GPT-2 Implememtation"""
	batch = tf.shape(logits)[0]
	sorted_logits = tf.sort(logits, direction='DESCENDING', axis=-1)
	cumulative_probs = tf.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)

	indices = tf.stack([
		tf.range(0, batch),
		tf.maximum(tf.reduce_sum(tf.cast(cumulative_probs <= p, tf.int32), axis=-1) - 1, 0),
	], axis=-1)

	min_values = tf.gather_nd(sorted_logits, indices)
	return tf.where(
		logits < min_values,
		tf.ones_like(logits) * -1e10,
		logits,
	)

In [ ]:
def sample_sequence(context=None, seq_len=256, temperature=1,top_k=8,top_p=8, nucleus_sampling=False):
  if context == None:
    print("Give some context to model.................")
    return
  prev_ctx = tf.expand_dims(tokenizer.encode(context), 0)
  for i in range(seq_len):
    logits, past = model(prev_ctx, training=False)
    print(logits)
    logits = logits[:, -1, :] / tf.cast(temperature, tf.float32)
    print(logits)
    logits = top_k_logits(logits, k=top_k)
    print(logits)
    if nucleus_sampling:
      logits = top_p_logits(logits, p=top_p)

    samples = tf.random.categorical(logits, num_samples=1, dtype=tf.int32)
    # print(samples)

    # print("shape.........")
    # print(tf.shape(output))
    # print(tf.shape(samples))
    output = tf.concat([prev_ctx, samples], axis=-1)
    prev_ctx = output
    # print(tf.shape(output))
    # print(output)

  # print("--------------------------")
  result = tf.squeeze(output, axis=0)
  pred = [int(i) for i in result]
  generated_seq = tokenizer.decode(pred[1:])
  return generated_seq


In [ ]:
seq = sample_sequence("def get_mean_and_stddevs(self, sites, rup, dists, imt, stddev_types):\n   ")
print(seq.ids)

ValueError: ignored